In [1]:
from pyspark.context import SparkContext
import pyspark.sql.functions as F
import time
from pyspark.sql import Window



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1603267417159_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spark.sql("use cdc")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [9]:
spark.sql("show tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
|     cdc|         contratti|      false|
|     cdc|           credito|      false|
|     cdc|          prodotti|      false|
|     cdc|punti_di_fornitura|      false|
|     cdc|          soggetti|      false|
+--------+------------------+-----------+

In [10]:
spark.sql("select count(*) from contratti").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|42728655|
+--------+

In [11]:
sqlContext = HiveContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_soggetti=sqlContext.table("datalake.l_orcl_admin_soggetti")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_soggetti.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- nazione_nascita: string (nullable = true)
 |-- rating_creditizio: string (nullable = true)
 |-- retention_value: string (nullable = true)
 |-- comune_residenza: string (nullable = true)
 |-- cognome: string (nullable = true)
 |-- eta_cliente: decimal(10,0) (nullable = true)
 |-- vas: string (nullable = true)
 |-- canale_contatto_preferenziale: string (nullable = true)
 |-- titolo_di_studio: string (nullable = true)
 |-- tipologia_cliente: string (nullable = true)
 |-- data_iscrizione_sportello_online: timestamp (nullable = true)
 |-- key_soggetti: decimal(10,0) (nullable = true)
 |-- nome: string (nullable = true)

In [14]:
windowSpec=Window.partitionBy(df_soggetti.nome,df_soggetti.cognome).\
orderBy(F.col("key_soggetti").desc())

deduplica_soggetti=df_soggetti.withColumn("rank",F.row_number().over (windowSpec)).filter (F.col("rank")==1)

deduplica_soggetti.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- nazione_nascita: string (nullable = true)
 |-- rating_creditizio: string (nullable = true)
 |-- retention_value: string (nullable = true)
 |-- comune_residenza: string (nullable = true)
 |-- cognome: string (nullable = true)
 |-- eta_cliente: decimal(10,0) (nullable = true)
 |-- vas: string (nullable = true)
 |-- canale_contatto_preferenziale: string (nullable = true)
 |-- titolo_di_studio: string (nullable = true)
 |-- tipologia_cliente: string (nullable = true)
 |-- data_iscrizione_sportello_online: timestamp (nullable = true)
 |-- key_soggetti: decimal(10,0) (nullable = true)
 |-- nome: string (nullable = true)
 |-- rank: integer (nullable = true)

In [15]:
deduplica_soggetti.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+---------------+--------------------+----------+-----------+----+-----------------------------+-----------------+-----------------+--------------------------------+------------+------+----+
|nazione_nascita|rating_creditizio|retention_value|    comune_residenza|   cognome|eta_cliente| vas|canale_contatto_preferenziale| titolo_di_studio|tipologia_cliente|data_iscrizione_sportello_online|key_soggetti|  nome|rank|
+---------------+-----------------+---------------+--------------------+----------+-----------+----+-----------------------------+-----------------+-----------------+--------------------------------+------------+------+----+
|         Italia|                C|           null|            Bagheria| Ardizzone|         58|null|                         null|Laurea magistrale|     Residenziale|                            null|    18354582|Abelie|   1|
|         Italia|                B|           null|              Ancona|   Augusta|         42|null|